In [4]:
import mod_sql

import imp
imp.reload()

In [ ]:
import pandas as pd

In [5]:
# mod_sql 통해서, Database 클래스 생성

db = mod_sql.Database("localhost", "root", "1234", "ubion7", 3306)

In [6]:
sql = """
    select
    `Country` as '국가',
    `Units Sold` as '판매개수',
    `Unit Price` as '가격',
    `Unit Cost` as '원가',
    (`Units Sold` * `Unit Price`) as '총수익',
    (`Units Sold` * `Unit Cost`) as '총비용',
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from
    `Sales Records`
"""

db.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


### Pandas 로도 위 셀과 같은 output 만들어보자


In [7]:
df = pd.read_csv('../csv/Sales Records.csv')
df.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [8]:
# df = df[['Country', 'Units Sold', 'Unit Price', 'Unit Cost']]
df.drop(['Region', 'Item Type', 'Sales Channel', 'Order Priority',\
         'Order Date', 'Order ID', 'Ship Date'], axis = 1, inplace = True)

In [9]:
df.columns = ['국가', '판매개수', '가격', '원가', '총수익', '총비용', '총이윤']

In [10]:
df

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


In [12]:
# (`Units Sold` * `Unit Price`) as '총수익',
#     (`Units Sold` * `Unit Cost`) as '총비용',
#     (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'

df['총수익zn'] = df['판매개수'] * df['가격']
df['총비용zn'] = df['판매개수'] * df['원가']
df['총이윤zn'] = df['총수익zn'] - df['총비용']

In [13]:
df.head(1)

,국가,판매개수,가격,원가,총수익,총비용,총이윤,총수익zn,총비용zn,총이윤zn
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.0,2920025.64,2353920.64,566105.0


### sql 에서 반올림


In [15]:
# python -> 반올림 : round(data, 몇째자리까지?)
sql = """
    select
    `Total Cost`,
    round(`Total Cost`, 1)
    from
    `Sales Records`
"""

db.sql_query(sql)

,Total Cost,"round(`Total Cost`, 1)"
0,2353920.64,2353920.6
1,34174.25,34174.2
2,592408.95,592409.0
3,3861014.82,3861014.8
4,110978.89,110978.9
...,...,...
9995,273298.63,273298.6
9996,147991.46,147991.5
9997,229507.32,229507.3
9998,3174991.14,3174991.1


## sql 그룹화

1. Country, Item Type 기준으로 그룹화
2. Unit Sold의 평균, 합계 생성 
3. Country 기준으로 오름차순 정렬


In [16]:
sql = """
    select
    `Country`,
    `Item Type`,
    AVG(`Units Sold`) as AVG,
    SUM(`Units Sold`) as SUM
    from
    `Sales Records`
    group by
    `Country`,
    `Item Type`
    order by
    `Country`
    ASC

"""

db.sql_query(sql)

,Country,Item Type,AVG,SUM
0,Afghanistan,Baby Food,1368.0000,1368
1,Afghanistan,Beverages,4793.3333,14380
2,Afghanistan,Cereal,6101.8889,54917
3,Afghanistan,Clothes,5977.4000,29887
4,Afghanistan,Cosmetics,3044.2500,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.0000,9640
2194,Zimbabwe,Office Supplies,4361.7500,17447
2195,Zimbabwe,Personal Care,9239.0000,18478
2196,Zimbabwe,Snacks,5438.0000,16314


Pandas로도 얼른 만들어보자

In [34]:
df = pd.read_csv('../csv/Sales Records.csv')

In [35]:
df = df[['Country', 'Item Type', 'Units Sold']]

In [33]:
# ZiNo
df.groupby(['Country', 'Item Type']).agg(['mean', 'sum'])

Units Sold       
                                    mean    sum
Country     Item Type                          
Afghanistan Baby Food        1368.000000   1368
            Beverages        4793.333333  14380
            Cereal           6101.888889  54917
            Clothes          5977.400000  29887
            Cosmetics        3044.250000  12177
...                                  ...    ...
Zimbabwe    Meat             9640.000000   9640
            Office Supplies  4361.750000  17447
            Personal Care    9239.000000  18478
            Snacks           5438.000000  16314
            Vegetables       6789.000000  33945

[2198 rows x 2 columns]

In [36]:
# Moon
df.groupby(['Country', 'Item Type']).agg(['mean', 'sum'])['Units Sold'].reset_index()

,Country,Item Type,mean,sum
0,Afghanistan,Baby Food,1368.000000,1368
1,Afghanistan,Beverages,4793.333333,14380
2,Afghanistan,Cereal,6101.888889,54917
3,Afghanistan,Clothes,5977.400000,29887
4,Afghanistan,Cosmetics,3044.250000,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.000000,9640
2194,Zimbabwe,Office Supplies,4361.750000,17447
2195,Zimbabwe,Personal Care,9239.000000,18478
2196,Zimbabwe,Snacks,5438.000000,16314


In [37]:
db.sql_close()

### dept table에서 부서의 이름 (dname)이 sales인 소속 사원의 정보 출력
1. dept table에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력 

In [164]:
# 1. ZiNo
_db = mod_sql.Database("localhost", "root", "1234", "ubion7", 3306)

In [42]:
sql = """
    select
    `DEPTNO`
    from
    `dept`
    where
    `DNAME` = 'SALES' 
"""

_db.sql_query(sql)

,DEPTNO
0,30.0


In [44]:
# 2. ZiNo

_sql = """
    select
    *
    from
    `emp`
    where
    `DEPTNO` = 30
""" 

_db.sql_query(_sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [50]:
# Moon : 1 & 2 한번에 -> 이중 쿼리문
# """안에는 들여쓰기 의미 없는 듯"""

sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = (
        select
        `deptno`
        from
        `dept`
        where
        `dname` = 'research'
    )
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
4,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [51]:
# {column명} in ('','') : 컬럼 안에 값이 ''또는 ''인 select한 컬럼 값 출력 
# 조건에 2개이상 값 넣으려면 in !!!

sql = """
    select
    `deptno`
    from
    `dept`
    where
    `dname` in ('sales', 'research')
"""

_db.sql_query(sql)

,deptno
0,20.0
1,30.0


In [55]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` in (
        select
        `deptno`
        from
        `dept`
        where
        `dname` in ('sales', 'research')       
    )
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


Pandas로 위 셀 output 형태 얼른 만들어보자

In [56]:
df1 = pd.read_csv('../csv/dept.csv')
df2 = pd.read_csv('../csv/emp.csv')

In [57]:
df1

,DEPTNO,DNAME,LOC
0,10.0,ACCOUNTING,NEW YORK
1,20.0,RESEARCH,DALLAS
2,30.0,SALES,CHICAGO
3,40.0,OPERATIONS,BOSTON


In [58]:
df2

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [96]:

# ZiNo
# df1.loc[(df1['DNAME'] == 'SALES') | (df1['DNAME'] == 'RESEARCH'), 'DEPTNO']
# 조건의 형태가 series 이면 and , or 을 기호로 쓰자 -> & ,|

# Moon -> isin() 도 가능띠 & tolist()로 series 결과값을 list로 만들어주자 -> 아래셀에서 활용하게 

flag = df1.loc[df1['DNAME'].isin(['RESEARCH', 'SALES']), 'DEPTNO'].tolist()
flag


[20.0, 30.0]

In [97]:
df2.loc[df2['DEPTNO'].isin(flag)].reset_index(drop = True)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


1. emp dataframe에서 JOB 컬럼의 데이터를 확인하여, 새로운 파생변수 
2. job_code : SALESMAN이면 1, MANAGER면 2, 나머지는 3으로 대입 


In [126]:
emp = pd.read_csv('../csv/emp.csv')
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [116]:
# Moon

# Case I : For문
# I-I : i in emp.index 

emp['job_code'] = 3

for i in emp.index : 
    if emp['JOB'][i] == 'SALESMAN' :
        emp.loc[i,'job_code'] = 1

    elif emp['JOB'][i] == 'MANAGER' :
        emp.loc[i, 'job_code'] = 2

    # else : 
    #     emp.loc[i, 'job_code'] = 3

emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [119]:
# Case I-II : for i in emp['JOB']

emp['job_code'] = 3
job_list = ['SALESMAN', 'MANAGER']

for i in job_list : 
    
    if i == 'SALESMAN' :
        emp.loc[emp['JOB'] ==i, 'job_code'] = 1
    elif i == 'MANAGER' :
        emp.loc[emp['JOB'] ==i, 'job_code'] = 2
    

emp
    

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [125]:
# Case II : apply()

# Case II-I : lambda

emp['job_code'] = emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else(2 if (x == 'MANAGER') else 3))

emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [127]:
# Case II-II : def function

def change(x) : 
    if x == 'SALESMAN' :
        result = 1
    elif x == 'MANAGER' : 
        result = 2
    else : 
        result = 3
    return result

emp['job_code'] = emp['JOB'].apply(change)
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [129]:
# Moon 위과정 w/ SQL
# CASE ~ END
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    CASE
        WHEN `JOB` = 'SALESMAN'
        THEN 1 
        WHEN `JOB` = 'MANAGER'
        THEN 2
        ELSE 3
    END as `JOB_CODE`
    from
    emp
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,JOB_CODE
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


In [131]:
# IF 문 SQL

sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    if (
        'JOB' = 'SALESMAN',
        1,
        if(
            `JOB` = 'MANAGER',
            2,
            3
        )
    ) as `job_code`
    from
    emp
    
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,3
2,7521.0,WARD,SALESMAN,3
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,3
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,3


1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000이상이면 High, 아니면 Low로 데이터 삽입


In [132]:
# ZiNo 1. Pandas

emp = pd.read_csv('../csv/emp.csv')

In [135]:
emp['H/L'] = emp['SAL'].apply(lambda x : 'High' if(x >= 2000) else('Low'))

emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,Low
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,Low
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,High
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,Low
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,High
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,High
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,High
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,Low


In [136]:
# ZiNo 2. SQL - IF문 <- 형태 무즈카시이

sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    `MGR`,
    `HIREDATE`,
    `SAL`,
    if (
        `SAL` >= 2000,
        'High',
        'Low'
    ) as  `H/L`
    from
    emp
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,Low
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,Low
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,High
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,Low
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,High
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,High
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,High
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,Low


In [140]:
# Moon SQL - CASE~END 문 

sql = """
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    CASE
        WHEN `SAL` >= 2000
        THEN 'High'
        ELSE 'Low'
    END as 'H/L'
    from
    emp
"""

_db.sql_query(sql)

,EMPNO,ENAME,SAL,H/L
0,7369.0,SMITH,800.0,Low
1,7499.0,ALLEN,1600.0,Low
2,7521.0,WARD,1250.0,Low
3,7566.0,JONES,2975.0,High
4,7654.0,MARTIN,1250.0,Low
5,7698.0,BLAKE,2850.0,High
6,7782.0,CLARK,2450.0,High
7,7788.0,SCOTT,3000.0,High
8,7839.0,KING,5000.0,High
9,7844.0,TURNER,1500.0,Low


### join

1. emp와 dept table 결합

In [141]:
# on ~ 다음에 오는 내용이 조건 

sql = """
    select
    *
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO


In [143]:
# 특정 컬럼만 뽑고싶다? 

sql = """
    select
    emp.EMPNO,
    emp.ENAME,
    emp.DEPTNO,
    dept.DNAME,
    dept.LOC
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

_db.sql_query(sql)

,EMPNO,ENAME,DEPTNO,DNAME,LOC
0,7369.0,SMITH,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,30.0,SALES,CHICAGO
2,7521.0,WARD,30.0,SALES,CHICAGO
3,7566.0,JONES,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,30.0,SALES,CHICAGO
5,7698.0,BLAKE,30.0,SALES,CHICAGO
6,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,20.0,RESEARCH,DALLAS
8,7839.0,KING,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,30.0,SALES,CHICAGO


In [144]:
# Pandas 로 Try it ! 


dept = pd.read_csv('../csv/dept.csv')
emp = pd.read_csv('../csv/emp.csv')


In [146]:
pd.merge(emp, dept, on= 'DEPTNO', how = 'left')[['EMPNO','ENAME','DEPTNO','DNAME','LOC']]

,EMPNO,ENAME,DEPTNO,DNAME,LOC
0,7369.0,SMITH,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,30.0,SALES,CHICAGO
2,7521.0,WARD,30.0,SALES,CHICAGO
3,7566.0,JONES,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,30.0,SALES,CHICAGO
5,7698.0,BLAKE,30.0,SALES,CHICAGO
6,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,20.0,RESEARCH,DALLAS
8,7839.0,KING,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,30.0,SALES,CHICAGO


In [149]:
# right로 join결합 해보자 
# 결과값 보면 DEPTNO 40이 NaN으로 떠
# 왜? right 결합이면 40은 떠야하자나 
    # select에서 가져오는 col이 emp.DEPTNO이기 때문이야
    # dept.DEPTNO 하면 뜨게 될거야

sql = """
    select
    emp.EMPNO,
    emp.ENAME,
    dept.DEPTNO,
    dept.DNAME,
    dept.LOC
    from
    emp
    right join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

_db.sql_query(sql)

,EMPNO,ENAME,DEPTNO,DNAME,LOC
0,7934.0,MILLER,10.0,ACCOUNTING,NEW YORK
1,7839.0,KING,10.0,ACCOUNTING,NEW YORK
2,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
3,7902.0,FORD,20.0,RESEARCH,DALLAS
4,7876.0,ADAMS,20.0,RESEARCH,DALLAS
5,7788.0,SCOTT,20.0,RESEARCH,DALLAS
6,7566.0,JONES,20.0,RESEARCH,DALLAS
7,7369.0,SMITH,20.0,RESEARCH,DALLAS
8,7900.0,JAMES,30.0,SALES,CHICAGO
9,7844.0,TURNER,30.0,SALES,CHICAGO


In [148]:
pd.merge(emp, dept, on= 'DEPTNO', how = 'right')[['EMPNO','ENAME','DEPTNO','DNAME','LOC']]

,EMPNO,ENAME,DEPTNO,DNAME,LOC
0,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
1,7839.0,KING,10.0,ACCOUNTING,NEW YORK
2,7934.0,MILLER,10.0,ACCOUNTING,NEW YORK
3,7369.0,SMITH,20.0,RESEARCH,DALLAS
4,7566.0,JONES,20.0,RESEARCH,DALLAS
5,7788.0,SCOTT,20.0,RESEARCH,DALLAS
6,7876.0,ADAMS,20.0,RESEARCH,DALLAS
7,7902.0,FORD,20.0,RESEARCH,DALLAS
8,7499.0,ALLEN,30.0,SALES,CHICAGO
9,7521.0,WARD,30.0,SALES,CHICAGO


### 문제 

- 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보를 출력


In [155]:
# ZiNo - sql


sql = """
    select
    *
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
    where
    `LOC` != 'NEW YORK'
    AND
    `LOC` != 'DALLAS'
"""
_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,30.0,SALES,CHICAGO


In [158]:
# MOON - sql
# 일단, 뉴욕, 댈러스가 아닌 부분만 꺼내서 사원번호만 필요하니까 사원번호만 꺼내자

sql = """
    select
    DEPTNO
    from
    dept
    where
        `LOC` not in ('NEW YORK', 'DALLAS')
"""


_db.sql_query(sql)


,DEPTNO
0,30.0
1,40.0


In [159]:
# Moon cont'd
# 위에서 30,40을 파악해서 넣거나

sql= """
    select
    *
    from 
    emp
    where
    DEPTNO = 30 or DEPTNO = 40
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [160]:
# Moon cont'd

sql= """
    select
    *
    from 
    emp
    where
    DEPTNO in (30, 40)
"""
_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [161]:
# Moon cont'd
# 30,40 을 직접 기입하지 않고 바로 가져오려면? 

sql= """
    select
    *
    from 
    emp
    where
    DEPTNO in (select DEPTNO from dept where LOC not in ('NEW YORK', 'DALLAS'))
"""
_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [162]:
# tran -> Workbench

## 테이블의 결합

1. 유니언 결합 (단순 행 결합)
2. 조인 결합

In [165]:
sql = """
    select
    *
    from
    tran_1
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
4995,T0000005108,210000,2019-06-15 02:42:41,HD315748
4996,T0000005109,150000,2019-06-15 03:36:16,HI215420
4997,T0000005110,50000,2019-06-15 03:44:06,IK880102
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758


In [166]:
sql = """
    select * from tran_1
    union
    select * from tran_2
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [167]:
sql = """
    select * from tran_d_1
    union
    select * from tran_d_2
"""

_db.sql_query(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [169]:
# tran_1 & tran_d_1 일단 두개 결합 띠 

sql = """
    select * from tran_1 left join tran_d_1 on tran_1.transaction_id = tran_d_1.transaction_id
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d_1.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0.0,T0000000113,S005,1.0
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1.0,T0000000114,S001,1.0
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2.0,T0000000115,S003,1.0
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3.0,T0000000116,S005,1.0
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4.0,T0000000117,S002,2.0
...,...,...,...,...,...,...,...,...
5238,T0000005108,210000,2019-06-15 02:42:41,HD315748,NaN,None,None,NaN
5239,T0000005109,150000,2019-06-15 03:36:16,HI215420,NaN,None,None,NaN
5240,T0000005110,50000,2019-06-15 03:44:06,IK880102,NaN,None,None,NaN
5241,T0000005111,210000,2019-06-15 04:14:06,IK074758,NaN,None,None,NaN


In [172]:
# tran 1&2, tran_d_1&2 총 4개 테이블 유니언 결합 

sql = """
    select * from 
    (select * from tran_1
    union
    select * from tran_2) as tran
    left join 
    (select * from tran_d_1
    union
    select * from tran_d_2) as tran_d
    on tran.transaction_id = tran_d.transaction_id
"""

_db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1


In [175]:
# Pandas

df1 = pd.read_csv('../csv/tran_1.csv')
df2 = pd.read_csv('../csv/tran_2.csv')
df3 = pd.read_csv('../csv/tran_d_1.csv')
df4 = pd.read_csv('../csv/tran_d_2.csv')

In [190]:
df12 = pd.concat([df1, df2], axis = 0, ignore_index= True)
df12

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [192]:
df34 = pd.concat([df3, df4], axis='rows', ignore_index= True)
df34

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [193]:
pd.merge(df12, df34,  on = 'transaction_id', how = 'left')

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


In [199]:
# merge 할 때 컬럼명 다른데 걔 기준 merge하고 싶다면? 

pd.merge(df12, df34,  left_on = 'transaction_id', right_on= 'transaction_id', how = 'left')

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


### 문자열이 포함되어 있는 경우 출력

In [201]:
# ENAME에 S가 앞, 뒤, 전체에 있는것만 뽑고싶다 -> S%%, %%S, %%S%% 로 표현 <- 파이썬에서 %하나쓰면 에러일수도

sql = """
   select
   *
   from
   emp
   where 
   `ENAME` like '%%S' 
"""

_db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
1,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
2,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


1. pandas 이용하여 emp.csv 에서 ENAME이 S로 시작하는 데이터를 출력
2. S로 끝나는 데이터를 출력
3. S가 포함되어 있는 데이터를 출력

In [202]:
df = pd.read_csv('../csv/emp.csv')
df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


In [246]:
df_0 = pd.DataFrame()
df_0['ENAME0'] = ''

df['ENAME'].apply(lambda x : x if (('S' in x[:]) ) else '')
df_0

,ENAME0
0,SMITH
1,
2,
3,JONES
4,
5,
6,
7,SCOTT
8,
9,


In [247]:
df_0 = pd.merge(df_0, df, left_on = 'ENAME0', right_on='ENAME', how = 'left').dropna(axis=0)
df_0

,ENAME0,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,SMITH,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,JONES,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,SCOTT,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,ADAMS,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,JAMES,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [249]:
# Moon
emp.loc[emp['ENAME'].apply(lambda x : x.startswith('S'))]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [250]:
emp.loc[emp['ENAME'].apply(lambda x : x.endswith('S'))]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [254]:
emp.loc[emp['ENAME'].str.contains('S')]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [263]:
# ZiNo
df.loc[df['ENAME'].apply(lambda x : 'S' in x)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [265]:
# Moon 다른 방법

emp.loc[emp['ENAME'].str.startswith('S')]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [266]:
# Moon : for 문

data = pd.DataFrame()
for i in emp.index : 
    if emp.loc[i, 'ENAME'].startswith('S') : 
        data = pd.concat([data, emp.loc[[i]]])

data

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0


In [268]:
# Moon : list & for문

_list = []

for i in emp.index : 
    if emp.loc[i, 'ENAME'].endswith('S') : 
        _list.append(i)

emp.loc[_list]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [275]:
#Moon : find()

# S로 시작 
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') == 0 )]

# S로 끝
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') == (len(x)-1))]

# S 중간
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') != -1)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [278]:
_db.sql_close()